##### 第一模块(导入包模块：必须运行）

In [1]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
# tk文件导入模块

##### 第二模块(文件处理模块：必须运行）

In [2]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")
        
def remote_select():
    print("以下文件可以调用，需要分析哪个文件？")
    print("-------------------------------------")
    index = 0
    list_item_temp = []
    for item in os.listdir():
        if ".xlsx" in item:
            index += 1
            print("[" + str(index) + "] " + item)
            list_item_temp.append(item)
    try:
        bash_pos = "/Users/dfuser/Desktop/目标文书目录/"
        file_code = int(input("需要导入哪个文件？(输入[]中的序号)"))
        final_pos = bash_pos + str(list_item_temp[file_code-1])
        table_ = pd.read_excel(final_pos)
        return table_
        print("表格导入成功，以下是表格预览")
        print("----------------------------")
    except Exception as e:
        print("导入错误")
        
def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该词频表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass

##### 第三模块(核心提取器模块：必须运行）

In [3]:
### 刑事的问题，必须要放刑事案件！！！
def find_keywords_in_courts_v3(li):
    list_key_expressions = []
    pattern_sentences = re.compile("([^。]*犯.*?罪[^。]*)。")
    for item in li:
        try:
            facts = re.findall(pattern_sentences,item)
            list_key_expressions.append(facts)
        except Exception as e:
            facts = None
            list_key_expressions.append(None)
    return list_key_expressions

def get_crimes_name(series):
    final_result_names = []
    pattern_limitation = re.compile("(犯.*?罪)")
    list_key_expressions = find_keywords_in_courts_v3(series) #table_final_2["法院判决"])
    list_key_expression_filter_1 = list(filter(lambda x:x != None,list_key_expressions))
    list_key_expression_filter_2 = list(filter(lambda x:x != [],list_key_expression_filter_1))
    for sentences in list_key_expression_filter_2:
        for sentence in sentences:
            final_result = re.search(pattern_limitation,sentence).group()
            if final_result != "犯罪":
                if len(final_result) <= 10:
                    final_result_names.append(final_result)
    crimes_table = pd.DataFrame([final_result_names,[1] * (len(final_result_names))]).T
    crimes_table_final = crimes_table.groupby(0).count().sort_values(by = 1,ascending = False)
    return crimes_table_final

def run_crimes_name(table):
    table_clean_dropna = table["判决结果"].dropna()
    table_final = get_crimes_name(table_clean_dropna)
    table_final.index.name = "罪名"
    table_final.columns = ["数量"]
    return table_final

#### 运行模块

In [4]:
if __name__ == '__main__': 
    flag_input = int(input("你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)"))
    if flag_input == 1:
        root = Tk() # 实例化TKinter窗口
        root.withdraw() # 隐藏TKinter窗口
        table_ready_to_eat = import_table(root)
    elif flag_input == 2:
        table_ready_to_eat = remote_select()
    else:
        logging.error("加载错误")
    
    table_final = run_crimes_name(table_ready_to_eat)
    save_model(table_final)

你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)2
以下文件可以调用，需要分析哪个文件？
-------------------------------------
[1] 夫妻债务 - 原表.xlsx
[2] ex1.xlsx
[3] 校园暴力 - 结构化重构.xlsx
[4] 校园暴力事件 - 原表.xlsx
[5] 正当防卫 - 原表.xlsx
[6] 夫妻债务.xlsx
[7] .~zhaiwu_regex_filter.xlsx.xlsx
[8] 职业打假人 - 原表.xlsx
需要导入哪个文件？(输入[]中的序号)3


KeyError: '判决结果'